In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime

def calc_risk_price(start_date, end_date, stablecoins, selected_stablecoin, volatility_dict=None, base_dict=None):
    # If no volatility is provided, default to the same volatility for all
    if volatility_dict is None:
        volatility_dict = {coin: 0.005 for coin in stablecoins}

    # If no base is provided, default to a base of 1 for all
    if base_dict is None:
        base_dict = {coin: 1.0 for coin in stablecoins}

    # List to store data
    data = []

    # Generate date range
    dates = pd.date_range(start=start_date, end=end_date, freq='D')
    random.seed(43)
    # Simulate data for each day and stablecoin
    for date in dates:
        for coin in stablecoins:
            # Fetch the specific volatility and base for the coin
            volatility = volatility_dict.get(coin, 0.005)  # Default to 0.005 if not provided
            base_value = base_dict.get(coin, 1.0)  # Default to 1.0 if not provided

            # Introduce random fluctuations around the specified base value for depegging
            price_variation = random.uniform(-volatility, volatility)
            price = base_value + price_variation
            data.append([date, coin, round(price, 4)])

    # Convert to dataframe
    df = pd.DataFrame(data, columns=['Date', 'Stablecoin', 'Price'])
    df_wide = df.pivot(index='Date', columns='Stablecoin', values='Price')
    # Calculate the average deviation for each stablecoin (risk factor)
    avg_dev = df_wide.apply(lambda x: np.abs(x - 1).mean(), axis=0)

    # Min-Max scaling for average deviation (scaled risk factor)
    dev_risk_scores = 0.01 + ((avg_dev - avg_dev.min()) / (avg_dev.max() - avg_dev.min())) * 0.98

    # Calculate volatility for each stablecoin
    vol = df_wide.apply(lambda x: np.sqrt(((x - x.mean()) ** 2).mean()))

    # Apply Min-Max scaling per stablecoin for volatility
    vola_risk_scores = 0.01 + ((vol - vol.min()) / (vol.max() - vol.min())) * 0.98

    # Calculate persistence for each stablecoin
    pers = df_wide.apply(lambda x: np.abs((((x[1:] - 1) * (x[:-1] - 1)).sum()) / (((x - 1) ** 2).sum())))

    # Min-Max scaling for persistence
    per_risk_scores = 0.01 + ((pers - pers.min()) / (pers.max() - pers.min())) * 0.98

    # Ensure case-insensitive lookup for stablecoin names
    stablecoin_name = next((coin for coin in vola_risk_scores.index if coin.lower() == selected_stablecoin.lower()), None)
    if stablecoin_name is None:
        raise ValueError(f"Stablecoin not found in the list: {selected_stablecoin}")

    # Retrieve the risk scores for the selected stablecoin
    v = vola_risk_scores[stablecoin_name]  # Volatility risk score for selected stablecoin
    dv = dev_risk_scores[stablecoin_name]  # Average deviation risk score for selected stablecoin
    ps = per_risk_scores[stablecoin_name]  # Persistence risk score for selected stablecoin

    # Calculate the price using the formula: p = BP * (1 + v) * (1 + dv) * (1 + ps) ###5000 minimum amouny
    price = (0.05 * 5000) * (1 + v) * (1 + dv) * (1 + ps)
    risk_score = np.cbrt(v * dv * ps)

    return price, risk_score


# Example usage
stablecoins = ['Satoshi', 'BIMA', 'Lorenzo', 'Bedrock', 'FDUSD', 'LstBTC', 'LBTC', 'BounceBit_BTC', 'USDe', 'TUSD']

volatility_for_coins = {
    'Satoshi': 0.043,
    'BIMA': 0.069,
    'Lorenzo': 0.057,
    'Bedrock': 0.525,
    'FDUSD': 0.078,
    'LstBTC': 0.251,
    'LBTC': 0.026,
    'BounceBit_BTC': 0.094,
    'USDe': 0.740,
    'TUSD': 0.661
}

base_for_coins = {coin: 1 for coin in stablecoins}

start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 10, 31)

# User input
cover_amount = float(input("Enter the cover amount: "))
stablecoin = input("Enter the stablecoin (Satoshi, BIMA, Lorenzo, Bedrock, FDUSD, LstBTC, LBTC, BounceBit_BTC, USDe, TUSD): ")

if stablecoin.lower() not in [coin.lower() for coin in stablecoins]:
    print(f"Invalid stablecoin. Please choose from {stablecoins}.")
else:
    # Calculate price
    price, risk_score = calc_risk_price(
        start_date=start_date,
        end_date=end_date,
        stablecoins=stablecoins,
        selected_stablecoin=stablecoin,
        volatility_dict=volatility_for_coins,
        base_dict=base_for_coins
    )
    print(f"The Risk Score for {stablecoin}: {risk_score:.2f}")
    print(f"Calculated premium for {stablecoin}: {round(5+(risk_score *15))}%")


The Risk Score for lbtc: 0.04
Calculated premium for lbtc: 6%


In [2]:
# Create a dictionary to store the results
results = {}

for coin in stablecoins:
    price, risk_score = calc_risk_price(
        start_date=start_date,
        end_date=end_date,
        stablecoins=stablecoins,
        selected_stablecoin=coin,
        volatility_dict=volatility_for_coins,
        base_dict=base_for_coins
    )
    # Calculate premium
    premium = round(5 + (risk_score * 15))

    # Add to results dictionary
    results[coin] = {"Risk Score": round(risk_score, 2), "Premium (%)": premium}

# Output the dictionary
print(results)


{'Satoshi': {'Risk Score': 0.06, 'Premium (%)': 6}, 'BIMA': {'Risk Score': 0.13, 'Premium (%)': 7}, 'Lorenzo': {'Risk Score': 0.09, 'Premium (%)': 6}, 'Bedrock': {'Risk Score': 0.77, 'Premium (%)': 17}, 'FDUSD': {'Risk Score': 0.17, 'Premium (%)': 8}, 'LstBTC': {'Risk Score': 0.42, 'Premium (%)': 11}, 'LBTC': {'Risk Score': 0.04, 'Premium (%)': 6}, 'BounceBit_BTC': {'Risk Score': 0.15, 'Premium (%)': 7}, 'USDe': {'Risk Score': 0.21, 'Premium (%)': 8}, 'TUSD': {'Risk Score': 0.89, 'Premium (%)': 18}}


In [ ]:
import json

# Save the dictionary to a JSON file
with open('stablecoin_risk_scores.json', 'w') as json_file:
    json.dump(results, json_file, indent=4)
print("Saved as JSON: stablecoin_risk_scores.json")
